In [27]:
import os
import requests
import time
import json
import re
from collections import defaultdict
from typing import Dict, List, Tuple, Optional

# LeetCode session information
LEETCODE_SESSION = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzb2NpYWxhY2NvdW50X3N0YXRlIjpbeyJuZXh0IjoiLyIsInByb2Nlc3MiOiJsb2dpbiIsInNjb3BlIjoiIiwiYXV0aF9wYXJhbXMiOiIifSwiUTM3ZERqblhWejBkIl0sIl9hdXRoX3VzZXJfaWQiOiI4NDAxMzM5IiwiX2F1dGhfdXNlcl9iYWNrZW5kIjoiZGphbmdvLmNvbnRyaWIuYXV0aC5iYWNrZW5kcy5Nb2RlbEJhY2tlbmQiLCJfYXV0aF91c2VyX2hhc2giOiI4NzYwZDgyNTA5MmQ4ZjVmNDQ4YzFlNDc2NjQ3ZjlhZjAzMzQ4MmFlYzQ3NjI0NWM3NDNlOWM0YWJkNDNmMWNkIiwiaWQiOjg0MDEzMzksImVtYWlsIjoiIiwidXNlcm5hbWUiOiJzZXJlbmUta2lsYnlncHAiLCJ1c2VyX3NsdWciOiJzZXJlbmUta2lsYnlncHAiLCJhdmF0YXIiOiJodHRwczovL2Fzc2V0cy5sZWV0Y29kZS5jbi9hbGl5dW4tbGMtdXBsb2FkL2RlZmF1bHRfYXZhdGFyLnBuZyIsInBob25lX3ZlcmlmaWVkIjp0cnVlLCJpcCI6IjE0LjE1My40LjE3NCIsIl90aW1lc3RhbXAiOjE3NjM0NDUwMDAuMDI2NjcyLCJleHBpcmVkX3RpbWVfIjoxNzY1OTk4MDAwLCJ2ZXJzaW9uX2tleV8iOjAsImRldmljZV9pZCI6IjY5MjUwNGE2MmMwN2IxMGZlZTlmMDk1MTc1NzRlNGZkIn0.8xZAQWDFVp2Fju1cKdliaY_HcgeI0IOUJRLRZeOfjas"
LEETCODE_CSRF = "tE0LHLreLPRzPliR5qP7OD8KbuWt548X"

class LeetCodeSubmitter:
    def __init__(self, solutions_dir: str, record_file: str = "0.txt"):
        """
        Initialize LeetCode submitter (enhanced retry mechanism)
        
        Args:
            solutions_dir (str): Directory containing C solutions
            record_file (str): Submission record file path
        """
        self.solutions_dir = solutions_dir
        self.record_file = record_file
        self.session = requests.Session()
        
        # Set request headers
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Content-Type': 'application/json',
            'X-CSRFToken': LEETCODE_CSRF,
            'Origin': 'https://leetcode.cn',
            'Referer': 'https://leetcode.cn/problems/'
        }
        
        # Set cookies
        self.session.cookies.set("LEETCODE_SESSION", LEETCODE_SESSION)
        self.session.cookies.set("csrftoken", LEETCODE_CSRF)
        
        # Initialize statistics
        self.total_problems = 0
        self.submitted_problems = 0
        self.accepted_problems = 0
        self.failed_problems = 0
        self.skipped_problems = 0
        self.problem_details = []
        
        # Load submission records
        self.submission_records = self._load_submission_records()
        print("LeetCode session initialized, ready to submit solutions...")
    
    def _load_submission_records(self) -> Dict[str, str]:
        """Load submission records from record file"""
        records = {}
        if os.path.exists(self.record_file):
            try:
                with open(self.record_file, 'r', encoding='utf-8') as f:
                    for line in f:
                        line = line.strip()
                        if line and ':' in line:
                            parts = line.split(':', 1)
                            problem_name = parts[0].strip()
                            status = parts[1].strip()
                            records[problem_name] = status
                print(f"Loaded {len(records)} submission records from {self.record_file}")
            except Exception as e:
                print(f"Error loading submission records: {str(e)}")
        return records
    
    def _save_submission_record(self, problem_name: str, status: str) -> None:
        """Save submission record to file"""
        try:
            with open(self.record_file, 'a', encoding='utf-8') as f:
                f.write(f"{problem_name}: {status}\n")
        except Exception as e:
            print(f"Error saving submission record: {str(e)}")
    
    def _get_problem_slug(self, filename: str) -> str:
        """Extract problem slug from filename"""
        # Example: "1.two-sum.c" -> "two-sum"
        # "789-escape-the-ghosts.c" -> "escape-the-ghosts"
        basename = os.path.basename(filename)
        name_without_ext = os.path.splitext(basename)[0]
        
        # Remove leading numbers and separators (support dot and hyphen)
        if '.' in name_without_ext:
            return name_without_ext.split('.', 1)[1]
        elif '-' in name_without_ext:
            # Keep hyphens in problem names (like "escape-the-ghosts")
            parts = name_without_ext.split('-', 1)
            if len(parts) > 1:
                return parts[1]
        return name_without_ext
    
    def _get_problem_id(self, slug: str) -> Optional[str]:
        """Get problem ID"""
        max_retries = 5  # Increase retry count
        retry_count = 0
        
        while retry_count < max_retries:
            try:
                # Get problem information
                graphql_url = "https://leetcode.cn/graphql/"
                graphql_payload = {
                    "operationName": "questionData",
                    "variables": {"titleSlug": slug},
                    "query": """
                        query questionData($titleSlug: String!) {
                            question(titleSlug: $titleSlug) {
                                questionId
                                questionFrontendId
                                title
                                titleSlug
                            }
                        }
                    """
                }
                
                response = self.session.post(
                    graphql_url,
                    headers=self.headers,
                    json=graphql_payload,
                    timeout=10
                )
                
                if response.status_code == 200:
                    data = response.json()
                    question_data = data.get("data", {}).get("question", {})
                    if question_data:
                        return question_data.get("questionId")
                    else:
                        print(f"Failed to get problem ID: abnormal response structure - {data}")
                else:
                    print(f"Failed to get problem ID: HTTP {response.status_code}")
                
            except Exception as e:
                print(f"Error getting problem ID: {str(e)}")
            
            # Wait before retry
            retry_count += 1
            if retry_count < max_retries:
                wait_time = 2 ** retry_count  # Exponential backoff
                print(f"Waiting {wait_time} seconds before retrying to get problem ID...")
                time.sleep(wait_time)
        
        print(f"Cannot get ID for problem '{slug}', skipping")
        return None
    
    def _submit_solution(self, problem_slug: str, solution_code: str) -> Tuple[str, str]:
        """Submit single solution (with retry mechanism)"""
        max_retries = 20  # Increase max retries to 20
        retry_count = 0
        
        while retry_count < max_retries:
            try:
                # Get problem ID
                problem_id = self._get_problem_id(problem_slug)
                if not problem_id:
                    return "Failed", "Cannot get problem ID"
                
                # Build submission data
                submit_url = f"https://leetcode.cn/problems/{problem_slug}/submit/"
                
                payload = {
                    "lang": "c",  # Changed from "cpp" to "c"
                    "question_id": problem_id,
                    "typed_code": solution_code
                }
                
                # Send submission request
                response = self.session.post(
                    submit_url,
                    headers=self.headers,
                    json=payload,
                    timeout=30
                )
                
                if response.status_code == 200:
                    result = response.json()
                    if "submission_id" in result:
                        submission_id = result["submission_id"]
                        print(f"Submission successful, submission ID: {submission_id}")
                        # Check submission result
                        return self._check_submission_result(submission_id)
                    else:
                        error_msg = "submission_id not found in submission response"
                        print(error_msg)
                elif response.status_code == 403:
                    error_msg = f"Submission failed: HTTP 403 Forbidden (attempt {retry_count+1}/{max_retries})"
                    print(error_msg)
                else:
                    error_msg = f"Submission failed: HTTP {response.status_code}"
                    print(error_msg)
                
            except Exception as e:
                error_msg = f"Error during submission: {str(e)}"
                print(error_msg)
            
            # Wait before retry (exponential backoff)
            retry_count += 1
            if retry_count < max_retries:
                wait_time = min(5 * retry_count, 60)  # Max wait 60 seconds
                print(f"Waiting {wait_time} seconds before retrying submission...")
                time.sleep(wait_time)
        
        return "Failed", f"Submission failed, reached maximum retry count ({max_retries})"
    
    def _check_submission_result(self, submission_id: str) -> Tuple[str, str]:
        """Check submission result"""
        max_retries = 30  # Maximum 30 retries
        retry_count = 0
        
        while retry_count < max_retries:
            try:
                # Query submission status
                check_url = f"https://leetcode.cn/submissions/detail/{submission_id}/check/"
                response = self.session.get(check_url, headers=self.headers, timeout=10)
                
                if response.status_code != 200:
                    return "Pending", f"Failed to check status: HTTP {response.status_code}"
                
                result = response.json()
                state = result.get("state", "PENDING")
                
                # Check if completed
                if state == "SUCCESS":
                    status = result.get("status_msg", "UNKNOWN")
                    runtime = result.get("status_runtime", "N/A")
                    return status, runtime
                
                # If still processing, wait and retry
                time.sleep(1)
                retry_count += 1
                
            except Exception as e:
                print(f"Error checking submission result: {str(e)}")
                time.sleep(2)
                retry_count += 1
        
        return "Timeout", "Exceeded maximum check attempts"
    
    def submit_all_solutions(self) -> None:
        """Submit all C solutions"""
        start_time = time.time()
        
        # Get all C solution files
        solution_files = [f for f in os.listdir(self.solutions_dir) 
                         if f.endswith('.c')]  # Changed from '.cpp' to '.c'
        solution_files.sort()  # Sort by filename
        self.total_problems = len(solution_files)
        
        if not solution_files:
            print("No C solution files found")
            return
        
        print(f"Found {self.total_problems} C solutions")
        print(f"Skipping problems with existing submission records...")
        
        # Submit each solution
        for i, filename in enumerate(solution_files):
            problem_name = os.path.splitext(filename)[0]
            problem_slug = self._get_problem_slug(filename)
            solution_path = os.path.join(self.solutions_dir, filename)
            
            print(f"\n{'='*50}")
            print(f"Processing problem ({i+1}/{self.total_problems}): {problem_slug} (filename: {filename})")
            
            # Check if submission record exists (skip any status)
            if problem_name in self.submission_records:
                record_status = self.submission_records[problem_name]
                print(f"Problem {problem_slug} already has submission record (status: {record_status}), skipping")
                self.skipped_problems += 1
                self.problem_details.append({
                    "problem": problem_slug,
                    "status": f"Skipped (Recorded: {record_status})",
                    "runtime": "N/A",
                    "filename": filename
                })
                continue
            
            try:
                # Read solution code
                with open(solution_path, 'r', encoding='utf-8') as f:
                    solution_code = f.read()
                
                # Submit solution (with retry)
                status, runtime = self._submit_solution(problem_slug, solution_code)
                
                # Update record
                self._save_submission_record(problem_name, status)
                self.submission_records[problem_name] = status
                
                # Update statistics
                self.submitted_problems += 1
                if status == "Accepted":
                    self.accepted_problems += 1
                
                # Save problem details
                self.problem_details.append({
                    "problem": problem_slug,
                    "status": status,
                    "runtime": runtime,
                    "filename": filename
                })
                
                # Print current status
                print(f"Submission result: {status}, Runtime: {runtime}")
                
                # Submission interval (avoid high request frequency)
                time.sleep(5)  # Increased to 5 seconds
                
            except Exception as e:
                error_msg = f"Error processing {filename}: {str(e)}"
                print(error_msg)
                self.problem_details.append({
                    "problem": problem_slug,
                    "status": "Error",
                    "runtime": "N/A",
                    "error": error_msg,
                    "filename": filename
                })
        
        # Print statistics after processing
        self.print_statistics(start_time)
    
    def print_statistics(self, start_time: float) -> None:
        """Print submission statistics"""
        elapsed_time = time.time() - start_time
        minutes, seconds = divmod(elapsed_time, 60)
        hours, minutes = divmod(minutes, 60)
        
        print("\n" + "="*50)
        print("LeetCode C Solutions Submission Report")
        print("="*50)
        print(f"Total problems: {self.total_problems}")
        print(f"Submitted problems: {self.submitted_problems}")
        print(f"Skipped problems: {self.skipped_problems}")
        print(f"Accepted problems: {self.accepted_problems}")
        print(f"Failed problems: {self.submitted_problems - self.accepted_problems}")
        
        # Status distribution
        status_counts = defaultdict(int)
        for problem in self.problem_details:
            status_counts[problem['status']] += 1
        
        print("\nStatus distribution:")
        for status, count in status_counts.items():
            print(f"- {status}: {count} problems")
        
        # Save detailed report to JSON file
        report_file = "submission_report.json"
        with open(report_file, 'w', encoding='utf-8') as f:
            json.dump({
                "total_problems": self.total_problems,
                "submitted_problems": self.submitted_problems,
                "skipped_problems": self.skipped_problems,
                "accepted_problems": self.accepted_problems,
                "problem_details": self.problem_details,
                "elapsed_time": elapsed_time
            }, f, indent=2, ensure_ascii=False)
        
        print(f"\nTotal time: {int(hours)} hours {int(minutes)} minutes {int(seconds)} seconds")
        print(f"Detailed report saved to: {report_file}")
        print("="*50)

def main():
    # Configuration parameters
    solutions_dir = "c_solutions"  # Changed from "cpp_solutions" to "c_solutions"
    record_file = "0.txt"          # Submission record file
    
    # Initialize submitter
    submitter = LeetCodeSubmitter(solutions_dir, record_file)
    
    try:
        submitter.submit_all_solutions()
    except KeyboardInterrupt:
        print("\nSubmission interrupted, printing current statistics...")
        submitter.print_statistics(time.time())

if __name__ == "__main__":
    main()

Loaded 2861 submission records from 0.txt
LeetCode session initialized, ready to submit solutions...
Found 3011 C solutions
Skipping problems with existing submission records...

Processing problem (1/3011): two-sum (filename: 1-two-sum.c)
Problem two-sum already has submission record (status: Accepted), skipping

Processing problem (2/3011): regular-expression-matching (filename: 10-regular-expression-matching.c)
Problem regular-expression-matching already has submission record (status: Accepted), skipping

Processing problem (3/3011): same-tree (filename: 100-same-tree.c)
Problem same-tree already has submission record (status: Accepted), skipping

Processing problem (4/3011): minimum-cost-to-merge-stones (filename: 1000-minimum-cost-to-merge-stones.c)
Problem minimum-cost-to-merge-stones already has submission record (status: Accepted), skipping

Processing problem (5/3011): grid-illumination (filename: 1001-grid-illumination.c)
Problem grid-illumination already has submission recor